In [10]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
from scipy.signal import *
from scipy import signal
import pandas as pd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn import svm

In [11]:
#Butter-Worth Bandpass Filter

def butterFil(signal):

    filt = butter(5, [0.15, 12], 'bp', fs = 40, output = 'sos')
    final = []

    for i in range(0, len(signal), 18):
        filtered = sosfilt(filt, signal[i:i+18])
        for j in filtered:
            final.append(j)

    #Normalizing the output
    ppgMax = max(final)
    ppgMin = min(final)
    final = (final - ppgMin) / (ppgMax - ppgMin)
    
    #Length of the signal
    t = np.linspace(0,60,len(final))
    
    ret = [final, t]
    return (ret)

In [12]:
#Derivative Filter

def derivativeFil(signal):

    sig = []
    der1 = []
    der2 = []
    
    #1st Derivative
    delta = 0.001
    for i in range(1, len(signal)-1):
        d = (signal[i+1]-signal[i-1])/(2*delta)
        der1.append(d)
    
    #2nd Derivative
    for i in range(1, len(der1)-1):
        d = (der1[i+1]-der1[i-1])/(delta**2)
        der2.append(d)
    
    return [der1, der2]

In [13]:
#Systolic Amplitude

def systolic_peak(signal):
    
    sys_peak = []
    for i in range(0, len(signal)-19, 19):
        sys_peak.append(max(signal[i:i+19]))
    
    #Average of the systolic peaks
    return sum(sys_peak)/len(sys_peak)

In [14]:
#Diastolic Amplitude

def diastolic_peak(signal):
    dia_peak = []
    flag = 0
    count = 0
    #Checking for a change in the slope of the signal from negative to positive
    for i in range(0, len(signal)-18, 18):
        for j in range(18):
            if signal[i+j+1]-signal[i+j]<0:
                flag = 1
            
            #When such a change is detected, the value of the signal at the next index is added to the list of diastolic peaks
            if signal[i+j+1]-signal[i+j]>0 and flag == 1:
                dia_peak.append(signal[i+j+1])
                flag = 0
                
    diaspeak = 0
    c = 0
    for i in range(0, len(dia_peak), 2):
        diaspeak += dia_peak[i]
        c += 1
        
    #Average of the diastolic peaks
    return diaspeak/c

In [15]:
#Pulse Interval

def pulse_interval(signal, time):
    times = []
    for i in range(0, len(signal)-19, 19):
        times.append(time[signal.index(max(signal[i:i+19]))])
    
    #Pulse intervals calculated by finding the difference between consecutive systolic peak times
    intervals = []
    for i in range(len(times)-1):
        intervals.append(times[i+1]-times[i])

    #Average pulse interval
    return sum(intervals)/len(intervals)

In [16]:
#Get Signal

def getSignal(video_name):

    count = 0
    cap = cv2.VideoCapture(video_name)
    mean = []

    while (cap.isOpened()):
        ret, frame = cap.read()
        if (not ret):
            break

        #Mean value of the red channel of each frame in the video
        red = frame[:,:,2]
        mean.append(np.mean(red))

        # 350 Frames
        if count == 350:
            break
        count += 1
    return mean

In [17]:
#Feature Extraction

def FeatureExtract(video, age):
    
    # To extract the mean value of the red channel of each frame in the video
    signal = getSignal(video)
    #Butterworth Filter
    new_PPGsignal = butterFil(signal)
    PPGsignal = new_PPGsignal[0]

    #2nd Order Derivative Features
    derFil = derivativeFil(PPGsignal)

    a = []
    b = []
    c = []
    d = []
    e = []

    for i in range(0, len(derFil[1])-18, 18):

        a.append(derFil[1][i+3])
        b.append(derFil[1][i+5])
        c.append(derFil[1][i+7])
        d.append(derFil[1][i+9])
        e.append(derFil[1][i+11])
    
    av = sum(a)/len(a)
    bv = sum(b)/len(b)
    cv = sum(c)/len(c)
    dv = sum(d)/len(d)
    ev = sum(e)/len(e)

    SysPeak = systolic_peak(PPGsignal)
    #Finding DiaStolic Amplitude
    DiaPeak = diastolic_peak(PPGsignal)
    #Finding the Pulse Interval
    t = np.linspace(0, 2.1, len(signal))
    pi = pulse_interval(PPGsignal.tolist(), t.tolist())
    #Finding the AI
    ai = DiaPeak/SysPeak
    #Finding the Adjusted AI
    adj_AI = 1 - ai
    #Finding the ATI/SP
    PiSp = pi/SysPeak
    #Finding ADJUSTED AI
    adj_AI = 1 - ai

    featureVector = [age, SysPeak, ai, adj_AI, ev/av, PiSp]
    return featureVector


In [18]:
#Dataframe contains equal number of diabetic and non-diabetic patient (38)

df1 = pd.read_csv("FinalFeatures_train_updated.csv")
Xtrain = df1.iloc[:, 1:7]
Ytrain = df1.iloc[:,-1]

In [25]:
def lda(Xtest):
    lda = LDA()
    lda.fit(Xtrain, Ytrain)
    ldares = lda.predict(pd.DataFrame(Xtest).T)
    print("LDA: ", ldares[0])
    return ldares[0]

In [26]:
def logistic_regression(Xtest):
    logreg = LogisticRegression(random_state=0).fit(Xtrain, Ytrain)
    reslr = logreg.predict(pd.DataFrame(Xtest).T)
    print("Logistic Regression: ", reslr[0])
    return reslr[0]

In [27]:
def support_vector_machine(Xtest):
    SVM = svm.SVC()
    SVM.fit(Xtrain, Ytrain)
    ressvm = SVM.predict(pd.DataFrame(Xtest).T)
    print("SVM: ", ressvm[0])
    return ressvm[00]

In [28]:
def decision_tree(Xtest):
    trees= tree.DecisionTreeClassifier()
    trees.fit(Xtrain, Ytrain)
    resdt = trees.predict(pd.DataFrame(Xtest).T)
    print("Decision Tree: ", resdt[0])
    return resdt[0]

In [29]:
#Recorded Video
# httpaddress = "http://192.168.43.1:8080"
httpaddress = 'Video Dataset/Yash.mp4'
age = 20

#Extract Patient's Features
patient = FeatureExtract(httpaddress, age)

In [30]:
decision = lda(patient) + support_vector_machine(patient) + decision_tree(patient) + logistic_regression(patient)

LDA:  0
SVM:  0
Decision Tree:  1
Logistic Regression:  0


c:\Users\yash2\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but LinearDiscriminantAnalysis was fitted with feature names
  warnings.warn(
c:\Users\yash2\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
c:\Users\yash2\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\yash2\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [31]:
#Final Decisions

if decision == 0:
    print("You are not diabetic.\n")
elif decision == 1 or decision == 2:
    print("Get yourself tested.\n")
else:
    print("You are diabetic.")

Get yourself tested.

